# Experiment 024: Morgan Fingerprints + Spange Descriptors

**Based on strategy:**
- CV-LB correlation is 0.994 - lower CV = lower LB
- Need CV < 0.0623 to beat exp_004
- Morgan fingerprints capture molecular structure not in Spange descriptors

**Approach:**
- Use exp_004 architecture (HGB for SM, ETR for Products) WITH TTA
- Add Morgan fingerprints (1024 bits) to feature set
- Combine: process + Spange + Morgan fingerprints

**TEMPLATE COMPLIANCE**: Last 3 cells are EXACTLY as template, NO cells after them.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# RDKit for molecular fingerprints
from rdkit import Chem
from rdkit.Chem import AllChem

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print("Setup complete")

Setup complete


In [2]:
# --- UTILITY FUNCTIONS ---
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature sets
SPANGE_DF = load_features('spange_descriptors')
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
SMILES_DICT = SMILES_DF['solvent smiles'].to_dict()
print(f"Spange: {SPANGE_DF.shape}, SMILES: {len(SMILES_DICT)} solvents")

Spange: (26, 13), SMILES: 26 solvents


In [3]:
# --- MORGAN FINGERPRINTS ---
def get_morgan_fingerprint(smiles, radius=2, n_bits=1024):
    """Get Morgan fingerprint as numpy array."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        # Handle mixtures like "O.CC#N"
        smiles_parts = smiles.split('.')
        mol = Chem.MolFromSmiles(smiles_parts[0])
        if mol is None:
            return np.zeros(n_bits)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    return np.array(fp)

# Pre-compute fingerprints for all solvents
MORGAN_FP_DICT = {}
for name, smiles in SMILES_DICT.items():
    MORGAN_FP_DICT[name] = get_morgan_fingerprint(smiles)
print(f"Computed Morgan fingerprints for {len(MORGAN_FP_DICT)} solvents")

Computed Morgan fingerprints for 26 solvents


In [4]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [5]:
# --- ENHANCED FEATURIZER (Spange + Morgan) ---
class EnhancedFeaturizer:
    """Featurizer combining Spange descriptors and Morgan fingerprints."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange = SPANGE_DF
        self.morgan = MORGAN_FP_DICT

    def _build_process_features(self, X):
        """Build process features with Arrhenius kinetics."""
        rt = X['Residence Time'].values.astype(np.float64).reshape(-1, 1)
        temp = X['Temperature'].values.astype(np.float64).reshape(-1, 1)
        
        # Arrhenius kinetic features
        temp_k = temp + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(rt + 1e-6)
        interaction = inv_temp * log_time
        
        return np.hstack([rt, temp, inv_temp, log_time, interaction])

    def _get_spange_features(self, X, flip=False):
        """Get Spange solvent features."""
        if self.mixed:
            A = self.spange.loc[X["SOLVENT A NAME"]].values
            B = self.spange.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                return B * (1 - pct) + A * pct
            else:
                return A * (1 - pct) + B * pct
        else:
            return self.spange.loc[X["SOLVENT NAME"]].values

    def _get_morgan_features(self, X, flip=False):
        """Get Morgan fingerprint features."""
        if self.mixed:
            A = np.array([self.morgan.get(s, np.zeros(1024)) for s in X["SOLVENT A NAME"]])
            B = np.array([self.morgan.get(s, np.zeros(1024)) for s in X["SOLVENT B NAME"]])
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                return B * (1 - pct) + A * pct
            else:
                return A * (1 - pct) + B * pct
        else:
            return np.array([self.morgan.get(s, np.zeros(1024)) for s in X["SOLVENT NAME"]])

    def featurize(self, X, flip=False):
        """Build combined features: process + Spange + Morgan."""
        process = self._build_process_features(X)
        spange = self._get_spange_features(X, flip)
        morgan = self._get_morgan_features(X, flip)
        
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)
            return np.hstack([process, pct, spange, morgan])
        return np.hstack([process, spange, morgan])

print("EnhancedFeaturizer defined")

EnhancedFeaturizer defined


In [6]:
# --- PER-TARGET MODEL WITH MORGAN FINGERPRINTS ---
class MorganPerTargetModel(BaseModel):
    """Per-target model with Spange + Morgan fingerprints.
    
    Architecture (same as exp_004):
    - SM: HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
    - Products: ExtraTreesRegressor (n_estimators=500, min_samples_leaf=2, max_depth=10)
    - TTA for full data predictions
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = EnhancedFeaturizer(mixed=(data=='full'))
        self.targets = ['Product 2', 'Product 3', 'SM']
        self.scaler = StandardScaler()
        self.models = {}

    def train_model(self, X_train, y_train):
        # Build features
        X_feat = self.featurizer.featurize(X_train)
        y = y_train.values
        
        if self.data_type == 'full':
            # Data augmentation with flipped features
            X_feat_flip = self.featurizer.featurize(X_train, flip=True)
            X_feat = np.vstack([X_feat, X_feat_flip])
            y = np.vstack([y, y])
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X_feat)
        
        # Train per-target models
        for i, target in enumerate(self.targets):
            y_target = y[:, i]
            
            if target == 'SM':
                model = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
            else:
                model = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    n_jobs=-1, random_state=42
                )
            
            model.fit(X_scaled, y_target)
            self.models[target] = model

    def predict(self, X):
        if self.data_type == 'full':
            # TTA for mixed solvents
            X_feat = self.featurizer.featurize(X, flip=False)
            X_feat_flip = self.featurizer.featurize(X, flip=True)
            
            X_scaled = self.scaler.transform(X_feat)
            X_scaled_flip = self.scaler.transform(X_feat_flip)
            
            preds = []
            for target in self.targets:
                p = (self.models[target].predict(X_scaled) + 
                     self.models[target].predict(X_scaled_flip)) / 2
                preds.append(p.reshape(-1, 1))
        else:
            X_feat = self.featurizer.featurize(X)
            X_scaled = self.scaler.transform(X_feat)
            
            preds = []
            for target in self.targets:
                p = self.models[target].predict(X_scaled)
                preds.append(p.reshape(-1, 1))
        
        final_preds = np.hstack(preds)
        final_preds = np.clip(final_preds, 0, 1)
        return torch.tensor(final_preds, dtype=torch.double)

print("MorganPerTargetModel defined")

MorganPerTargetModel defined


In [7]:
# Quick validation test
print("Testing MorganPerTargetModel...")
X_test, Y_test = load_data("single_solvent")

errors = []
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_test, Y_test)):
    if i >= 5: break
    model = MorganPerTargetModel(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    solvent = test_X['SOLVENT NAME'].iloc[0]
    print(f"Single Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nMorgan model quick test MAE: {np.mean(errors):.4f}")

Testing MorganPerTargetModel...


Single Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1688


Single Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.1274


Single Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0327


Single Fold 3 (Acetonitrile): MAE = 0.0699


Single Fold 4 (Acetonitrile.Acetic Acid): MAE = 0.1223

Morgan model quick test MAE: 0.1042


In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MorganPerTargetModel(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:03,  3.29s/it]

2it [00:06,  3.31s/it]

3it [00:09,  3.25s/it]

4it [00:12,  3.22s/it]

...

21it [01:11,  3.34s/it]

22it [01:14,  3.31s/it]

23it [01:17,  3.28s/it]

24it [01:20,  3.27s/it]

24it [01:20,  3.37s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MorganPerTargetModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:06,  6.12s/it]

2it [00:11,  5.71s/it]

3it [00:17,  5.66s/it]

4it [00:22,  5.61s/it]

...

10it [00:55,  5.52s/it]

11it [01:01,  5.60s/it]

12it [01:06,  5.63s/it]

13it [01:12,  5.65s/it]

13it [01:12,  5.59s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [11]:
# Calculate CV score
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Single solvent CV
single_preds = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
single_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_single, Y_single)):
    single_true.append(test_Y.values)
single_true = np.vstack(single_true)
single_mae = np.mean(np.abs(single_preds - single_true))
print(f"Single solvent CV MAE: {single_mae:.4f}")

# Full data CV
full_preds = submission_full_data[['target_1', 'target_2', 'target_3']].values
full_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full)):
    full_true.append(test_Y.values)
full_true = np.vstack(full_true)
full_mae = np.mean(np.abs(full_preds - full_true))
print(f"Full data CV MAE: {full_mae:.4f}")

# Combined
combined_mae = (single_mae + full_mae) / 2
print(f"\nCombined CV MAE: {combined_mae:.4f}")
print(f"exp_004 CV: 0.0623")
print(f"Difference: {combined_mae - 0.0623:.4f}")

Single solvent CV MAE: 0.0771
Full data CV MAE: 0.0990

Combined CV MAE: 0.0881
exp_004 CV: 0.0623
Difference: 0.0258
